# Simple QA Examples in FB2M & FB5M

Ensure that the examples facts in the SimpleQuestions dataset are present in the FB2M & FB5M KG. Otherwise those examples are unanswerable.

In [5]:
import sys
sys.path.insert(0, '../../')
import pandas as pd
from tqdm import tqdm_notebook
tqdm_notebook().pandas()

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

In [6]:
from lib.connect import get_connection 

connection = get_connection()
cursor = connection.cursor()

In [10]:
from lib.simple_qa import load_simple_qa 

df_train, df_dev, df_test = load_simple_qa(dev=True, train=True, test=True)
df_train[:5]

,subject,relation,object,question
0,04whkz5,book/written_work/subjects,01cj3p,what is the book e about
1,0tp2p24,music/release_track/release,0sjc7c1,to what release does the release track cardiac...
2,04j0t75,film/film/country,07ssc,what country was the film the debt from
3,0ftqr,music/producer/tracks_produced,0p600l,what songs have nobuo uematsu produced?
4,036p007,music/release/producers,0677ng,Who produced eve-olution?


In [14]:
from IPython.display import display, Markdown
from lib.data import FB2M_KG_TABLE

def check_fact_invariant(df, kg=FB2M_KG_TABLE):
    n_failed = 0.0
    for _, row in tqdm_notebook(df.iterrows(), total=df.shape[0]):
        sql = """SELECT count(*)
        FROM {kg}
        WHERE object_mid='{object_mid}'
        AND relation='{relation}'
        AND subject_mid='{subject_mid}'
        """.format(kg=kg,
                   object_mid=row['object'],
                   relation=row['relation'],
                   subject_mid=row['subject'])

        cursor.execute(sql)
        n_rows = cursor.fetchone()[0]
        if n_rows == 0:
            print('\n[%d] FACT NOT FOUND' % n_failed)
            print('Question: %s' % (row['question']))
            print('SQL: %s' % (sql))
            n_failed += 1
            

    
    display(Markdown(
        '### Numbers\n' + 
        'Facts in not found in KG\n' + 
        'Failed %%%f [%d of %d]' % ((n_failed / df.shape[0]) * 100, n_failed, df.shape[0])
    ))

In [15]:
check_fact_invariant(df_dev)

HBox(children=(IntProgress(value=0, max=10845), HTML(value='')))


[0] FACT NOT FOUND
Question: the film mom and dad is what genre? 
SQL: SELECT count(*)
        FROM fb_two_kg
        WHERE object_mid='07s9rl0'
        AND relation='film/film/genre'
        AND subject_mid='07gkcf'
        

[1] FACT NOT FOUND
Question: what is the name of a professional basketball player?
SQL: SELECT count(*)
        FROM fb_two_kg
        WHERE object_mid='0kg1cyw'
        AND relation='people/profession/people_with_this_profession'
        AND subject_mid='02h664x'
        



### Numbers
Facts in not found in KG
Failed %0.018442 [2 of 10845]

In [11]:
check_fact_invariant(df_test)

HBox(children=(IntProgress(value=0, max=21687), HTML(value='')))


[0] FACT NOT FOUND
Question: what are the names of russian basketball players?
SQL: SELECT count(*)
        FROM fb_two_kg
        WHERE object_mid='03y9cqy'
        AND relation='people/profession/people_with_this_profession'
        AND subject_mid='02h664x'
        

[1] FACT NOT FOUND
Question: What is a track on the release in paris?
SQL: SELECT count(*)
        FROM fb_two_kg
        WHERE object_mid='0fr432l'
        AND relation='music/release/track'
        AND subject_mid='0g1vb16'
        

[2] FACT NOT FOUND
Question: What is the last time released under?
SQL: SELECT count(*)
        FROM fb_two_kg
        WHERE object_mid='02lx2r'
        AND relation='music/album/release_type'
        AND subject_mid='07kg0mk'
        

[3] FACT NOT FOUND
Question: which basketball player is from bulgaria
SQL: SELECT count(*)
        FROM fb_two_kg
        WHERE object_mid='0jwydj2'
        AND relation='people/profession/people_with_this_profession'
        AND subject_mid='02h664x'
   

### Numbers
Facts in not found in KG
Failed %0.032277 [7 of 21687]

In [12]:
check_fact_invariant(df_train)

HBox(children=(IntProgress(value=0, max=75910), HTML(value='')))


[0] FACT NOT FOUND
Question: what kind of movie is the world
SQL: SELECT count(*)
        FROM fb_two_kg
        WHERE object_mid='07s9rl0'
        AND relation='film/film/genre'
        AND subject_mid='07m_vq'
        

[1] FACT NOT FOUND
Question: What region was the best released in?
SQL: SELECT count(*)
        FROM fb_two_kg
        WHERE object_mid='0345h'
        AND relation='music/release/region'
        AND subject_mid='0318_03'
        

[2] FACT NOT FOUND
Question: what type of release is the best?
SQL: SELECT count(*)
        FROM fb_two_kg
        WHERE object_mid='02lx2r'
        AND relation='music/album/release_type'
        AND subject_mid='01hh6cr'
        

[3] FACT NOT FOUND
Question: who is the artist on when the last time?
SQL: SELECT count(*)
        FROM fb_two_kg
        WHERE object_mid='054yql'
        AND relation='music/album/artist'
        AND subject_mid='0w9c45l'
        

[4] FACT NOT FOUND
Question: is the italian an art film or documentary 
SQL: S

### Numbers
Facts in not found in KG
Failed %0.019760 [15 of 75910]

## Discussion

FB5M and FB2M do not have a triple for 24 and 36 examples in SimpleQuestions dataset respectively. Those examples are impossible to answer. They should be removed in future iterations of the dataset.